# Detectando objetos com YOLO v4 e OpenCV

## Visão geral

No notebook anterior, utilizamos o framework Darknet para realizar a detecção de objetos em imagens com o You only look once (YOLO). Entretanto, também é possível empregar o módulo cv2, que corresponde à implementação do OpenCV em Python, para executar a mesma tarefa, sem a necessidade de compilar (buildar) o framework Darknet.

In [ ]:
#